## **Conformal Prediction using Energy Hospital Load**

One example of the NP data is the electricity consumption of a hospital in SF. It has hourly reservation for the entire year of 2015.

In [1]:
!python --versions

unknown option --versions
usage: python [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.


### Library Setup
Requires Python 3.9+

Install from scratch

In [2]:
# !git clone https://github.com/ourownstory/neural_prophet.git
# !cd neural_prophet
# !pip install -e ".[dev]"

### Extract Data and Split Data into Train and Test


**Extract Data From GitHub**

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')

from neuralprophet import NeuralProphet, set_log_level, set_random_seed

In [4]:
data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
file = 'energy/SF_hospital_load.csv'
# file = 'air_passengers.csv'

In [5]:
data_df = pd.read_csv(data_location + file)

In [6]:
# data_df.head(5)

In [7]:
# data_df.tail(5)

**Split data into train, val, cal, and test in that order**

Do we need to instantiate a NP model `m` in order to split the df into train and test? If so, does the NP params make any diff to the outcome of this split? I'm presuming no atm.

In [8]:
train_df, test_df = NeuralProphet().split_df(data_df, freq='H', valid_p = 1.0/16)
train_df.shape, test_df.shape

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.989% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


((8213, 2), (547, 2))

In [9]:
print(f"Train set time range:    {train_df['ds'].min()} - {train_df['ds'].max()}")
print(f"Test set time range:     {test_df['ds'].min()} - {test_df['ds'].max()}")

Train set time range:    2015-01-01 01:00:00 - 2015-12-09 05:00:00
Test set time range:     2015-12-09 06:00:00 - 2016-01-01 00:00:00


### Create Folds using CV Splits and Fit

In [10]:
freq = "H"
random_seed = 0
B_values = [10, 20, 30, 40]  # Number of bootstraps and number of k-folds
val_fold_pct = 0.04          # Validation fold size % of entire input
val_cov_pct = 0.8            # Overall validation set coverage %
val_fold_size = None         # Fixed validation fold size % of entire fold (hence train fold size is 1 - val_fold_size)
                             # Default is None for Extending Window/Rolling Origin, (0, 1) for Fixed Sliding Window/Blocked CV 

In [11]:
quantile_lo = 0.05
quantile_hi = 0.95
quantile_lo_str = str(quantile_lo*100)
quantile_hi_str = str(quantile_hi*100)
quantiles = [quantile_lo, quantile_hi]
n_lags = 3*24
params = {  # m1 by default, uncomment for m3
    'growth': 'off',
    'yearly_seasonality': False,
    'weekly_seasonality': False,
    'daily_seasonality': False,
    'n_lags': n_lags,
    'ar_reg': 1,
    'learning_rate': 0.01,
    'quantiles': quantiles,
}

In [12]:
cols = ['ds', 'y', 'yhat1', 'residual1', f'yhat1 {quantile_lo_str}%', f'yhat1 {quantile_hi_str}%']
cal_forecast_dict = defaultdict(pd.DataFrame)

In [ ]:
%%time
for B in B_values:
    # Split original training data into folds with sub-training and sub-calibration sets each
    folds = NeuralProphet().crossvalidation_split_df(
        train_df,
        freq=freq,
        k=B,
        fold_pct=val_fold_pct,
        fold_overlap_pct=(B*val_fold_pct - val_cov_pct)
    )

    # Train a bootstrap model on each sub-training set and predict on sub-calibration set per fold 
    for train_fold_df, cal_fold_df in folds:
        if val_fold_size:
            train_fold_len = int(len(cal_fold_df) * ((1-val_fold_size)/val_fold_size))
            train_fold_df = train_fold_df[-train_fold_len: ]
        # print(f"  - Train start: {train_fold_df.ds.min()}, Train end: {train_fold_df.ds.max()}")
        # print(f"  - Cal start:   {cal_fold_df.ds.min()}, Cal end:   {cal_fold_df.ds.max()}")
        # print(f"  - Train shape:   {train_fold_df.shape}, Cal shape:   {cal_fold_df.shape}")
        m1 = NeuralProphet(**params)
        set_random_seed(random_seed)
        metrics = m1.fit(train_fold_df, freq=freq, minimal=True)
        cal_forecast = m1.predict(cal_fold_df)
        cal_forecast_dict[B] = pd.concat([cal_forecast_dict[B], cal_forecast[cols]], ignore_index=True)

    # Aggregate the prediction for each sub-calibration row for all folds
    cal_forecast_dict[B] = cal_forecast_dict[B].groupby('ds').mean().reset_index()

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.988% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.973% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 133
Epoch[133/133]: 100%|██████████| 133/133 [00:31<00:00,  4.22it/s]
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.695% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.696% of the data.
INFO - (NP.df_utils._infer_frequency) - 

In [ ]:
def plot_nonconformity_scores(scores, q, method):
    plt.plot(scores, label="score")
    plt.axhline(y=q, color="r", linestyle="-", label=f"q1={round(q, 2)}")
    plt.xlabel("Sorted Index")
    plt.ylabel("Nonconformity Score")
    plt.title(f"{method} Nonconformity Score with q")
    plt.legend()
    plt.show();

In [ ]:
alpha = 0.1
qhat_df = pd.DataFrame(index=B_values)

In [ ]:
enbpi_qhats = []
for B in B_values:
    enbpi_noncon_scores = np.abs(cal_forecast_dict[B]['residual1'].values)
    enbpi_noncon_scores = enbpi_noncon_scores[~pd.isnull(enbpi_noncon_scores)]  # remove NaN values
    enbpi_noncon_scores.sort()
    # print(enbpi_noncon_scores.shape)

    # get the q-hat index and value
    enbpi_qhat_idx = int(len(enbpi_noncon_scores)*alpha)
    enbpi_qhat = enbpi_noncon_scores[-enbpi_qhat_idx]
    method = f'EnbPI_B{B}'
    plot_nonconformity_scores(enbpi_noncon_scores, enbpi_qhat, method)
    enbpi_qhats.append(enbpi_qhat)

qhat_df['enbpi'] = enbpi_qhats

In [ ]:
enbcqr_scoring_func = (
    lambda row: [None, None]
    if row[f"yhat1 {quantile_lo_str}%"] is None or row[f"yhat1 {quantile_hi_str}%"] is None
    else [
        max(row[f"yhat1 {quantile_lo_str}%"] - row["y"], row["y"] - row[f"yhat1 {quantile_hi_str}%"]),
        0 if row[f"yhat1 {quantile_lo_str}%"] - row["y"] > row["y"] - row[f"yhat1 {quantile_hi_str}%"] else 1,
    ]
)

enbcqr_qhats = []
for B in B_values:
    enbcqr_scores_df = cal_forecast_dict[B].apply(enbcqr_scoring_func, axis=1, result_type="expand")
    enbcqr_scores_df.columns = ["scores", "arg"]
    enbcqr_noncon_scores = enbcqr_scores_df["scores"].values
    enbcqr_noncon_scores = enbcqr_noncon_scores[~pd.isnull(enbcqr_noncon_scores)]  # remove NaN values
    enbcqr_noncon_scores.sort()
    print(enbcqr_noncon_scores.shape)

    # get the q-hat index and value
    enbcqr_qhat_idx = int(len(enbcqr_noncon_scores)*alpha)
    enbcqr_qhat = enbcqr_noncon_scores[-enbcqr_qhat_idx]
    method = f'EnbCQR_B{B}'
    plot_nonconformity_scores(enbcqr_noncon_scores, enbcqr_qhat, method)
    enbcqr_qhats.append(enbcqr_qhat)

qhat_df['enbcqr'] = enbcqr_qhats

Get forecasts and *EnbPI* and *EnbCQR* conformal prediction interval of OOS test_df

In [ ]:
cutoff = 7*24 # 0
x_size = 12
y_size = 6
figsize = (x_size, y_size)
plotting_backend = 'default'

In [ ]:
enbpi_forecast_dict = defaultdict(pd.DataFrame)
enbcqr_forecast_dict = defaultdict(pd.DataFrame)

In [ ]:
method = 'enbpi'
for B in B_values:
    m1.q_hats = [qhat_df.loc[B, method]]
    m1.conformal_method = 'naive'
    m1.quantile_lo = None
    m1.quantile_hi = None
    enbpi_forecast = m1.predict(test_df)
    # enbpi_forecast.head()
    enbpi_forecast_dict[B] = enbpi_forecast
    fig = m1.highlight_nth_step_ahead_of_each_forecast(1) \
            .plot(enbpi_forecast[-cutoff:], figsize=figsize, plotting_backend=plotting_backend)  # [-cutoff:]

In [ ]:
method = 'enbcqr'
for B in B_values:
    m1.q_hats = [qhat_df.loc[B, method]]
    m1.conformal_method = 'cqr'
    m1.quantile_lo = quantile_lo_str
    m1.quantile_hi = quantile_hi_str
    enbcqr_forecast = m1.predict(test_df)
    # enbcqr_forecast.head()
    enbcqr_forecast_dict[B] = enbcqr_forecast
    fig = m1.highlight_nth_step_ahead_of_each_forecast(1) \
            .plot(enbcqr_forecast[-cutoff:], figsize=figsize, plotting_backend=plotting_backend)  # [-cutoff:]

### Evaluate Performance

- **interval_width**: The average prediction interval, or q_hat multiplied by 2 because it is static or non-adaptive. Also the *efficiency* metric.
- **miscoverage_rate**: The actual miscoverage error rate on the OOS test set. Also the *validity* metric.

For the both metrics, the lower the number, the better the performance.



In [ ]:
B_values_str = [f'B_{B}' for B in B_values]
conformal_methods = ['enbpi_tscv', 'enbcqr_tscv']
eval_df = pd.DataFrame(columns=pd.MultiIndex.from_product([B_values_str, conformal_methods, ['interval_width','miscoverage_rate']]))
eval_df = eval_df.reset_index().rename(columns={'index': 'model'})

Enter in the model names

In [ ]:
eval_df['model'] = ['m3']

Enter in the **interval_width** (*efficiency* metric)

In [ ]:
# Calculate EnbCQR interval width function
def calc_enbcqr_iw(fcast_df, qr_lo, qr_hi, qhat):
    qr_lo_mean = fcast_df['yhat1'].mean() - fcast_df[f'yhat1 {qr_lo}%'].mean()
    qr_hi_mean = fcast_df[f'yhat1 {qr_hi}%'].mean() - fcast_df['yhat1'].mean()
    return qr_lo_mean + qr_hi_mean + qhat*2

In [ ]:
for B in B_values:
    # enbpi interval width
    enbpi_iw = [qhat_df.loc[B, 'enbpi']*2]
    eval_df[f'B_{B}', 'enbpi_tscv', 'interval_width'] = enbpi_iw

    # EnbCQR interval width
    enbcqr_iw = calc_enbcqr_iw(enbcqr_forecast, quantile_lo_str, quantile_hi_str, qhat_df.loc[B, 'enbcqr'])
    # cqr_iw1 = calc_cqr_iw(cqr_forecast1, quantile_lo_str, quantile_hi_str, cqr_qhat1)
    # cqr_iw2 = calc_cqr_iw(cqr_forecast2, quantile_lo_str, quantile_hi_str, cqr_qhat2)
    # cqr_iw3 = calc_cqr_iw(cqr_forecast3, quantile_lo_str, quantile_hi_str, cqr_qhat3)
    # cqr_iw4 = calc_cqr_iw(cqr_forecast4, quantile_lo_str, quantile_hi_str, cqr_qhat4)
    enbcqr_iw = [enbcqr_iw]
    # cqr_iw = [cqr_iw1, cqr_iw2, cqr_iw3, cqr_iw4]
    eval_df[f'B_{B}', 'enbcqr_tscv', 'interval_width'] = enbcqr_iw

Enter in the **miscoverage rate** (*validity* metric)

In [ ]:
# Calculate EnbPI miscoverage rate function
def calc_enbpi_mr(fcast_df):
    n_covered = fcast_df.apply(lambda row: bool(row['yhat1 - qhat1'] <= row['y'] <= row['yhat1 + qhat1']), axis=1)
    coverage_rate = n_covered.sum() / len(fcast_df)
    return 1 - coverage_rate

# Calculate EnbCQR and EnbCQR Advanced miscoverage rate function
def calc_enbcqr_mr(fcast_df, qr_lo, qr_hi):
    n_covered = fcast_df.apply(lambda row: bool(row[f'yhat1 {qr_lo}% - qhat1'] <= row['y'] <= row[f'yhat1 {qr_hi}% + qhat1']), axis=1)
    coverage_rate = n_covered.sum() / len(fcast_df)
    return 1 - coverage_rate

In [ ]:
for B in B_values:
    # enbpi miscoverage rate
    enbpi_mr = calc_enbpi_mr(enbpi_forecast_dict[B])
    # enbpi_mr1 = calc_enbpi_mr(enbpi_forecast1)
    # enbpi_mr2 = calc_enbpi_mr(enbpi_forecast2)
    # enbpi_mr3 = calc_enbpi_mr(enbpi_forecast3)
    # enbpi_mr4 = calc_enbpi_mr(enbpi_forecast4)
    enbpi_mr = [enbpi_mr]
    # enbpi_mr = [enbpi_mr1, enbpi_mr2, enbpi_mr3, enbpi_mr4]]
    eval_df[f'B_{B}', 'enbpi_tscv', 'miscoverage_rate'] = enbpi_mr

    # CQR miscoverage rate
    enbcqr_mr = calc_enbcqr_mr(enbcqr_forecast_dict[B], quantile_lo_str, quantile_hi_str)
    # cqr_mr1 = calc_cqr_mr(cqr_forecast1, quantile_lo_str, quantile_hi_str)
    # cqr_mr2 = calc_cqr_mr(cqr_forecast2, quantile_lo_str, quantile_hi_str)
    # cqr_mr3 = calc_cqr_mr(cqr_forecast3, quantile_lo_str, quantile_hi_str)
    # cqr_mr4 = calc_cqr_mr(cqr_forecast4, quantile_lo_str, quantile_hi_str)
    enbcqr_mr = [enbcqr_mr]
    # cqr_mr = [cqr_mr1, cqr_mr2, cqr_mr3, cqr_mr4]
    eval_df[f'B_{B}', 'enbcqr_tscv', 'miscoverage_rate'] = enbcqr_mr

Show evaluation dataframe

In [ ]:
eval_df

**Summary**: 

CQR outputs narrower prediction *interval width* than Naive while maintaining slightly better *miscoverage rate* (for the advanced AR models m4 and m4). The complex m4 model performs the best for *interval_width* while the simple m1 performs the best for actual *miscoverage rate*, hovering around *alpha*. However, that is because it has also by far the highest *interval_width*. m3 and m4 models have actual *miscoverage rates* are roughly twice the amount as the intended *alpha* of 0.1, which is consistent with the conformal prediction literature.